In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Activation, Layer, Dense
from packages.Utility.CNN.Callbacks import SaveModel, ValidateModel, StopTraining
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams, RotatedGaborParams, ChannelizeGaborParams

Using TensorFlow backend.


# Inisialisasi / Persiapan

In [2]:
fold_index = 0
dataset_dir = 'dataset/CF323'
trained_dir = 'trained/CF323 CF[1.2]'

# Memuat Dataset

In [3]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [4]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)

# Mendefinisikan Model

In [5]:
kernel1 = GaborFilterBanks(ChannelizeGaborParams())
kernel2 = GaborFilterBanks(ChannelizeGaborParams())
kernel3 = GaborFilterBanks(ChannelizeGaborParams())
kernel4 = GaborFilterBanks(ChannelizeGaborParams())
kernel5 = GaborFilterBanks(ChannelizeGaborParams())
kernel6 = GaborFilterBanks(ChannelizeGaborParams())

model = Sequential()

c1 = Conv2D(32, name = 'c1', kernel_size = 3, kernel_initializer = kernel1, padding='same', input_shape = (32, 32, 3))

c2 = Conv2D(32, name = 'c2', kernel_size = 3, kernel_initializer = kernel2, padding = 'same')
mp2 = MaxPool2D(pool_size = 2)

c3 = Conv2D(64, name = 'c3', kernel_size = 3, kernel_initializer = kernel3, padding = 'same')

c4 = Conv2D(64, name = 'c4', kernel_size = 3, kernel_initializer = kernel4, padding = 'same')
mp4 = MaxPool2D(pool_size = 2, name = 'mp4')

c5 = Conv2D(128, name = 'c5', kernel_size = 3, kernel_initializer = kernel5, padding = 'same')

c6 = Conv2D(128, name = 'c6', kernel_size = 3, kernel_initializer = kernel6, padding = 'same')
mp6 = MaxPool2D(pool_size = 2, name = 'mp6')

# Layer 1
model.add(c1)

# Layer 2
model.add(c2)
model.add(mp2)
 
# Layer 3
model.add(c3)

# Layer 4
model.add(c4)
model.add(mp4)
 
# Layer 5
model.add(c5)

# Layer 6
model.add(c6)
model.add(mp6)
 
model.add(Flatten(name = 'flatten'))
          
# Fully Connected
model.add(Dense(1024, name = 'd1', activation = 'sigmoid'))
model.add(Dense(10, name = 'output', activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.


# Pelatihan

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = model.fit(X_train, Y_train_onehot, epochs = 100, verbose = 1, batch_size = 128, callbacks=[val_model, save_model, stop_train], initial_epoch = 10)

Instructions for updating:
Use tf.cast instead.
Epoch 11/100
22500/22500 [==============================] - 122s 5ms/step - loss: 0.0864 - acc: 0.2250
Accuracy : 0.27066666666666667
Model Saved
New best accuracy : 0.27066666666666667
Epoch 12/100
22500/22500 [==============================] - 154s 7ms/step - loss: 0.0832 - acc: 0.2811
Accuracy : 0.3033777777777778
Model Saved
New best accuracy : 0.3033777777777778
Epoch 13/100
22500/22500 [==============================] - 155s 7ms/step - loss: 0.0817 - acc: 0.3014
Accuracy : 0.31893333333333335
Model Saved
New best accuracy : 0.31893333333333335
Epoch 14/100
22500/22500 [==============================] - 153s 7ms/step - loss: 0.0807 - acc: 0.3148
Accuracy : 0.32097777777777775
Model Saved
New best accuracy : 0.32097777777777775
Epoch 15/100
22500/22500 [==============================] - 158s 7ms/step - loss: 0.0803 - acc: 0.3202
Accuracy : 0.3230222222222222
Model Saved
New best accuracy : 0.3230222222222222
Epoch 16/100
22500/22500 [